# Step 2 - Recover train network from train schedules

In this notebook we recover the graph of the train network of Belgium from the data of railway schedules, geographical railway track data and stations GPS coordinates. This relies on the positional accuracy of datasets. 

In [1]:
import json
import geopandas as gp
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta, time
from tqdm.notebook import tqdm
import networkx as nx
from collections import defaultdict
import pickle
import os
from itertools import cycle

# Technical functions

In [2]:
def get_date_range(month_start, year_start, month_end, year_end):
    month_range = list(range(1,13))
    cycle_month_range = cycle(month_range)
    while True:
        current_month = next(cycle_month_range)
        if current_month == month_start:
            break
    date_tuples = []
    year = year_start
    while True:
        if current_month < 10:
            date_tuples.append(("0"+str(current_month), str(year)))
        else:
            date_tuples.append((str(current_month), str(year)))
        if year == year_end and current_month == month_end:
            break
        current_month = next(cycle_month_range)
        if current_month == 1:
            year += 1
    return date_tuples


def upload_schedule(in_file):
    def date_hook(json_dict):
        for (key, value) in json_dict.items():
            for index_list_entry in range(len(value)):
                for index_entry, v in enumerate(value[index_list_entry]):
                    try:
                        json_dict[key][index_list_entry][index_entry] = datetime.strptime(
                            v, "%Y-%m-%d %H:%M:%S"
                        )
                    except:
                        pass
        return json_dict

    with open(in_file, "r") as in_f:
        return json.load(in_f, object_hook=date_hook) 

# Upload data

#### Passenger trains schedules

We use this data to decide whether two stations can be linked together. Provide your custom start dates to have coarser or more precise result.

In [5]:
start_month = 9
start_year = 2019
end_month = 9
end_year = 2019

total_path_list = []
total_files_list = []

date_range = get_date_range(start_month, start_year, end_month, end_year)
base_in_dir = Path("./infrabel_data_json/")
for month, year in date_range:
    data_dir = os.path.join(base_in_dir, "-".join([year, month]))
    files_list = sorted(os.listdir(data_dir))
    path_list = [os.path.join(data_dir, f) for f in files_list]
    total_path_list += path_list
    total_files_list += files_list
print(total_files_list)

['2019-09-01', '2019-09-02', '2019-09-03', '2019-09-04', '2019-09-05', '2019-09-06', '2019-09-07', '2019-09-08', '2019-09-09', '2019-09-10', '2019-09-11', '2019-09-12', '2019-09-13', '2019-09-14', '2019-09-15', '2019-09-16', '2019-09-17', '2019-09-18', '2019-09-19', '2019-09-20', '2019-09-21', '2019-09-22', '2019-09-23', '2019-09-24', '2019-09-25', '2019-09-26', '2019-09-27', '2019-09-28', '2019-09-29', '2019-09-30']


In [6]:
schedule_loaded = []
for day_file, upload_path in tqdm(list(zip(total_files_list, total_path_list))):
    schedule_loaded.append((day_file, upload_schedule(upload_path)))

  0%|          | 0/30 [00:00<?, ?it/s]

# Recover the network from schedules

### Project all intra-country train track records onto the set of stations

The schedules contain records of train passage through stations on its way in a form of a consecutive list. We imply that two consecutive stations can be reached and put a link between them.

In [7]:
H = nx.Graph()

for day, day_schedule in tqdm(schedule_loaded):
    for train, schedule in day_schedule.items():

        first_time = True
        for row1, row2 in zip(schedule, schedule[1:]):
            previous_station = row1[0]
            current_station = row2[0]
            
            if first_time:
                previous_id = row1[-3]
                line_dep = row1[-1]
                if not H.has_node(previous_station):
                    H.add_node(previous_station, count = 1)
                first_time = False
            current_id = row2[-3]
            if not H.has_node(current_station):
                H.add_node(current_station, count = 1)
            else:
                H.nodes[current_station]["count"] += 1
            
            line_dep = row1[-1]
            line_arr = row2[-2]
            if not H.has_edge(previous_station, current_station):
                H.add_edge(previous_station, current_station, count = 1)
            else:
                H[previous_station][current_station]["count"] += 1

for e1,e2 in H.edges():
    H[e1][e2]["count"] /= len(schedule_loaded)
for u in H.nodes():
    H.nodes[u]["count"] /= len(schedule_loaded)
    
print(nx.info(H))

  0%|          | 0/30 [00:00<?, ?it/s]

Graph with 669 nodes and 1164 edges


In [9]:
node_counts = sorted([(dd["count"], u) for u,dd in H.nodes(data = True)], key = lambda x: x[0])
edge_counts = sorted([(dd["count"], e1,e2) for e1,e2,dd in H.edges(data = True)], key = lambda x: x[0])

nodes_small_number_trains = [u for d, u in node_counts if d < 5]

### Get and remove all garbage stations

We will delete all stations which are not passenger train stations, such as garages, wash stations, etc. They have appropriate suffixes. Also there is a shunting yard at SCHAARBEEK, which we delete as well.

In [10]:
bad_suffix_stations = []
suffixes = ["WIJKSPOOR", "SAS", "GARAGE", "CARWASH", "AFWISSELING", "FORMATION", 
            "GOEDEREN", "BUNDEL", "BUNDELS", "FAISCEAU", "MARCHANDISES","WIJKBUNDEL", 
            "A.T.", "T.W.", "RELAIS", "PERRON", "WIJKSBUNDEL"]
for u in tqdm(H.nodes()):
    if isinstance(u, int):
        continue
    for v in H.nodes():
        if isinstance(v, int):
            continue
        trim_v = v.split(" ")[0]
        split_v = trim_v.split("-")
        if len(split_v) > 1:
            ending = split_v.pop(-1)
            beginning = "-".join(split_v)
            if beginning == u:
                if ending in suffixes:
                    bad_suffix_stations.append(v)
# add schaarbeek constellation
possible_schaarbeeks = ["SCHAARBEEK", "SCHAARBEEK-JOSAPHAT"]
for v in tqdm(H.nodes()):
    if isinstance(v, int):
        continue
    if "SCHAARBEEK" in v:
        if v not in possible_schaarbeeks and v not in bad_suffix_stations:
            bad_suffix_stations.append(v)

  0%|          | 0/669 [00:00<?, ?it/s]

  0%|          | 0/669 [00:00<?, ?it/s]

### Remove the non garbage stations 

We remove the garbage stations from the list of all nodes in the graph.

In [11]:
remaining_stations = set(H.nodes()) - set(bad_suffix_stations) - set(nodes_small_number_trains)
len(remaining_stations)

580

### Store geo positions of remaining stations

Supplementary data is downloaded from here: https://infrabel.opendatasoft.com/explore/dataset/operationele-punten-van-het-newterk/information/. This is a file with all operational points in the Belgian railway network ans their coordinates. We select only stations (fr. "Point d'arrêt", "Gare") and store their coordinates ss attributes on the network nodes. 

In [13]:
stops_file = "./supplementary_data/points-operationnels-du-reseau.csv"
stat_pd = pd.read_csv(stops_file, header = 0, sep = ";")

coordinates_dict = {}

stat_stations = stat_pd[stat_pd["Classification"].isin(["Point d'arrêt", "Gare"])]

for index, row in tqdm(stat_stations.iterrows(), total = len(stat_stations)):
    station_name = row["Abréviation LST NL complète"]
    if (station_name not in coordinates_dict):
        coords = row["Geo Point"].split(",")
        X,Y = float(coords[1]), float(coords[0])
        coordinates_dict[row["Abréviation LST NL complète"]] = (X,Y)
        
# station_names, station_points = zip(*list(coordinates_dict.items()))

  0%|          | 0/670 [00:00<?, ?it/s]

### Recover the position of missing stations

The dataset turns out to be incomplete, therefore we recover the missing coordinates using the spring layout. We fix the known coordinates apriori. 

In [15]:
node_pos = {}
fixed_nodes = []
for u in H.nodes():
    if u in coordinates_dict:
        node_pos[u] = coordinates_dict[u]
        fixed_nodes.append(u)
    else:
        node_pos[u] = (4,50)
        
pos = nx.spring_layout(H, pos = node_pos, fixed = fixed_nodes, iterations = 50, k = 1/10000)

for u in H.nodes():
    H.nodes[u]["lon"] = pos[u][0]
    H.nodes[u]["lat"] = pos[u][1]

In [16]:
node_positions = {}
for u in remaining_stations:
    if u in coordinates_dict:
        node_positions[u] = coordinates_dict[u]
    else:
        node_positions[u] = list(pos[u])

## Save the dict with station coordinates

In [18]:
intermediate_data_dir = "./intermediate_data/"
os.makedirs(intermediate_data_dir, exist_ok=True)
nodes_with_coordinates_filepath = os.path.join(
    intermediate_data_dir, "remaining_nodes_with_coordinates.pkl"
)
pickle.dump(node_positions, open(nodes_with_coordinates_filepath, "wb"))